# 使用Apriori算法来发现频繁集

In [1]:
def loadDataSet():#create a simple dataset for testing below functions
	return [[1,3,4],[2,3,5],[1,2,3,5],[2,5]]

#create candidate itemsets
def createC1(dataSet):
	C1 = []						#candidate itemset of size one 存储所有不重复的项值
	for transaction in dataSet:	#遍历数据集中的所有交易记录
		for item in transaction:#遍历记录中的每一个项
			if not [item] in C1:#如果某个物品项没有在C1中出现，则将其添加到C1中
				C1.append([item])#add a list containing just one item
	C1.sort()					#对大列表进行排序
	return list(map(frozenset,C1))#将每个单元素列表映射到frozenset()，最后返回frozenset的列表

#generate L1 from C1
#Ck:a list of candidate sets
def scanD(D, Ck, minSupport):	#数据集、候选项集列表Ck，最小支持度
	ssCnt = {}					#empty dictionary
	for tid in D:				#go over all the transactions in the dataset
		for can in Ck:			#go over all the candidate sets in Ck
			if can.issubset(tid):
				if not can in ssCnt:
					ssCnt[can] = 1
				else:
					ssCnt[can] += 1#if the sets of Ck are part of the dataset, increase the count
									#in the dictionary, the set is the key in the dictionary
	numItems = float(len(D))#size of dataset
	retList = []
	supportData = {}#support values for frequent itemsets
	for key in ssCnt:#goes over every element in the dictionary and measures the support.
		support = ssCnt[key] / numItems#support
		if support >= minSupport:
			#print("retList:",retList)
			retList.insert(0,key)#add the key that meet the min support to retList
			#print("retList_insert:",retList)
		supportData[key] = support#return supportData
	return retList, supportData

In [2]:
dataSet = loadDataSet()
print(dataSet)
C1=createC1(dataSet)
print(C1)
D = list(map(set,dataSet))
print(D)
L1,supportData0 = scanD(D,C1,0.5)
print(L1)

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]
[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]


In [3]:
#take a list of freq itemsets Lk and the size of the itemsets k to produce Ck.
def aprioriGen(Lk, k):#Lk:frequent itemsets; k:the size of Lk
	retList = []
	lenLk = len(Lk)
	for i in range(lenLk):
		for j in range(i+1,lenLk):
			L1 = list(Lk[i])[:k-2]
			L2 = list(Lk[j])[:k-2]
			L1.sort()
			L2.sort()
			if L1 == L2:
				retList.append(Lk[i] | Lk[j])#combine
	return retList

#give a dataset and a support number then generate a list of candidate itemsets
def apriori(dataset, minSupport = 0.5):
	C1 = createC1(dataset)
	D = list(map(set,dataset))
	L1, supportData = scanD(D,C1,minSupport)
	L = [L1]
	k = 2
	while (len(L[k-2]) > 0):#to find L2,L3,...
		Ck = aprioriGen(L[k-2], k)
		Lk, supK = scanD(D,Ck,minSupport)#get Lk from Ck
		supportData.update(supK)
		L.append(Lk)
		k += 1
	return L, supportData

In [4]:
L,supportData = apriori(dataSet)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [5]:
supportData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

In [6]:
L[0]

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [7]:
L[1]

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]

In [8]:
L[2]

[frozenset({2, 3, 5})]

In [9]:
L[3]

[]

In [10]:
aprioriGen(L[0],2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

In [11]:
L,supportData = apriori(dataSet,minSupport = 0.7)
L

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

In [12]:
supportData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5}

In [13]:
aprioriGen(L[0],3)

[]

In [14]:
aprioriGen(L[0],1)

[frozenset({2, 5}), frozenset({3, 5}), frozenset({2, 3})]

In [15]:
aprioriGen(L[0],0)

[frozenset({2, 5}), frozenset({3, 5}), frozenset({2, 3})]

In [16]:
#generate a list of rules with confidence values
#three inputs:
#L: a list of freq itemsets
#supportData: a dictionary of support data for the itemsets
#minConf: a minimum confidence threshold
def generateRules(L, supportData, minConf = 0.7):
	bigRuleList = []#store the rules
	for i in range(1, len(L)):
		for freqSet in L[i]:#loop over every freq itemset in L and create a list of single-item sets
			H1 = [frozenset([item]) for item in freqSet]#H1 for each freq itemset
			if (i > 1):#get only sets with two or more items for merging
				rulesFromConseq(freqSet, H1, supportData, bigRuleList,minConf)
			else:#calculate the confidence
				calcConf(freqSet, H1, supportData, bigRuleList, minConf)
	return bigRuleList

#calculate the confidence and find out which rules meet the min confidence
def calcConf(freqSet, H, supportData, brl, minConf = 0.7):
	prunedH = []#list for the rules that meet the min confidence
	for conseq in H:#iterate over all the itemsets in H and calculate the conf
		conf = supportData[freqSet] / supportData[freqSet - conseq]#calc conf
		if conf >= minConf:#check meet or not
			print(freqSet - conseq,'-->',conseq,'conf:',conf)#display if meet min conf
			brl.append((freqSet - conseq,conseq,conf))
			prunedH.append(conseq)
	return prunedH

#merging
def rulesFromConseq(freqSet, H, supportData, brl, minConf = 0.7):
	m = len(H[0])#the size of the itemsets in H
	if (len(freqSet) > (m + 1)):#if the freq set is large enough to have subsets of size m removed
		Hmp1 = aprioriGen(H, m + 1)#generate combinations of the items in H without repeating and stored in Hmp1
		Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)#Hmp1 holds all the possible rules


In [17]:
L,suppData=apriori(dataSet,minSupport = 0.5)
rules = generateRules(L,suppData,minConf = 0.7)
L

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0


[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [18]:
rules

[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [19]:
rules = generateRules(L,suppData,minConf = 0.5)
rules

frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666


[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

### 示例：发现毒蘑菇的相似特征

In [20]:
mushDatSet = [line.split() for line in open('E:\\python\\machinelearning\\MLDownloads\\machinelearninginaction\\Ch11\\mushroom.dat').readlines()]
Lm,msupportData = apriori(mushDatSet,minSupport = 0.3)
for item in Lm[0]:
	if item.intersection('2'):# find intersection('2') in Lm[0]
		print(item)

frozenset({'2'})


In [21]:
for item in Lm[1]:
	if item.intersection('2'):
		print(item)

frozenset({'28', '2'})
frozenset({'2', '53'})
frozenset({'2', '23'})
frozenset({'2', '34'})
frozenset({'2', '36'})
frozenset({'2', '59'})
frozenset({'2', '63'})
frozenset({'2', '67'})
frozenset({'2', '76'})
frozenset({'2', '85'})
frozenset({'2', '86'})
frozenset({'2', '90'})
frozenset({'93', '2'})
frozenset({'2', '39'})


In [22]:
for item in Lm[3]:
	if item.intersection('2'):
		print(item)

frozenset({'28', '59', '2', '34'})
frozenset({'28', '2', '34', '63'})
frozenset({'28', '85', '2', '34'})
frozenset({'28', '86', '2', '34'})
frozenset({'28', '90', '2', '34'})
frozenset({'28', '2', '34', '39'})
frozenset({'28', '2', '34', '53'})
frozenset({'28', '59', '2', '63'})
frozenset({'28', '85', '59', '2'})
frozenset({'28', '85', '2', '63'})
frozenset({'28', '90', '85', '2'})
frozenset({'28', '85', '2', '39'})
frozenset({'28', '85', '2', '53'})
frozenset({'28', '86', '59', '2'})
frozenset({'28', '86', '2', '63'})
frozenset({'28', '86', '85', '2'})
frozenset({'28', '90', '86', '2'})
frozenset({'28', '86', '2', '39'})
frozenset({'28', '86', '2', '53'})
frozenset({'28', '90', '59', '2'})
frozenset({'28', '90', '2', '39'})
frozenset({'28', '59', '2', '39'})
frozenset({'63', '28', '2', '39'})
frozenset({'28', '90', '2', '53'})
frozenset({'28', '2', '53', '39'})
frozenset({'85', '2', '34', '53'})
frozenset({'86', '2', '34', '53'})
frozenset({'90', '2', '34', '53'})
frozenset({'2', '34'